In [28]:
import requests
from bs4 import BeautifulSoup

In [29]:
URL_BASE = 'https://www.cfcprodutivo.com.br'
URL_BUSCA = URL_BASE + '/framework/admin/ajax/'

In [30]:
s = requests.session()
URL_LOGIN = 'https://cfcprodutivo.com.br/framework/admin/ajax/login.php'

payload = {'login': 'cfcadm',
            'senha': 'abc123'}

login = s.request('POST', URL_LOGIN, data=payload)

In [31]:

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://www.cfcprodutivo.com.br/framework/admin/listar/alunos',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    'iDisplayStart': '0',
    'iDisplayLength': '100',
    'iSortCol_0': '1',
    'sSortDir_0': 'desc',
}

response = s.get(
    'https://www.cfcprodutivo.com.br/framework/admin/ajax/ajaxListaClientesAlunos.php',
    params=params,
    headers=headers,
)

if 'douglas-ferreira-alves' in response.text:
    print('ACHOU')

ACHOU


In [65]:
soap = BeautifulSoup(response.content, 'html.parser')
links = soap.findAll('a')

for link in links:
    url_form = link['href'].replace('\\', '').replace('"', '')
    response_page = s.request("GET", URL_BASE + url_form)
    break
    

In [66]:
# soap = BeautifulSoup(response_page.content, 'html.parser')
# table_infos = soap.find("div", {"id": "fichaDoAluno"})
# for linha in table_infos.findAll("div", {"class": "row-fluid"}):
#     print(linha)
#     break

In [67]:
import re 
id = int(re.search(r'\d+', response_page.url).group())

URL_FICHA = URL_BUSCA + 'containerTabClientesAlunosVerFicha.php?'

In [68]:
params = {
    'idAluno': id
}
response_ficha = s.request("GET", URL_FICHA, params=params)

soap = BeautifulSoup(response_ficha.content, 'html.parser')
table_infos = soap.find("div", {"id": "fichaDoAluno"})
ficha = dict()
for linha in table_infos.findAll("div", {"class": "control-group"}):
    ficha.update(
        {
            linha.label.text.strip(): linha.find("div", {"class": "controls"}).text.strip()
        })



In [69]:
URL_FINANCEIRO = URL_BUSCA + 'containerTabClientesAlunosVerMovimentacoes.php'

params = {
    'idAluno': id
}
response_ficha = s.request("GET", URL_FINANCEIRO, params=params)

soap = BeautifulSoup(response_ficha.content, 'html.parser')
financeiro = dict()
nome_coluna = soap.thead.findAll('th')
tds = soap.tbody.findAll('td')
for index, nome in enumerate(nome_coluna[0:-2]):
    financeiro.update(
        {
            nome.text.strip(): tds[index].text.strip()
        }
    )



In [96]:
import json
f = soap.find('div', {'class': 'btn-group right'}).strong
r = json.loads(f)

In [101]:
json.load(r)

AttributeError: 'str' object has no attribute 'read'

In [102]:
f

'"{Totais (entradas): Contratado: R$ 2.100,00,  | \\u00a0\\nPago: R$ 2.100,00,  | \\u00a0\\nPendente: R$ 0,00}"'

In [103]:
financeiro

{'Descrição': 'Contrato - Serviços: 1° HABILITAÇÃO AB - CARTÃO (1)',
 'Tipo': 'Entrada (receber)',
 'Usuário': 'Gisele',
 'Data de cadastro': '23/01/2023 13:11:25',
 'Valor total': 'R$ 2.100,00',
 'Já quitado': 'R$ 2.100,00',
 'Parcelas': '1/1',
 'Parcelas em atraso': '0',
 'Forma de pagamento': 'Cartão de Crédito'}